# MitoTNT: Mitochondrial Temporal Network Tracking

**MitoTNT** is a Python-based pipeline for the tracking, visualization, and dynamic analysis of 4D mitochondrial network data.

It is built upon mitochondria segmentation provided by MitoGraph, and visualization engine provided by ChimeraX.  

MitoTNT is written by Zichen (Zachary) Wang (ziw056@ucsd.edu), with the help from people in the [Johannes Schöneberg lab](https://www.schoeneberglab.org/) at UCSD.


# Installation

Software requirements:
- **[Jupyter Notebook](https://jupyter.org/)** or get it from **[Anaconda](https://www.anaconda.com/products/distribution)**

- **[MitoGraph](https://github.com/vianamp/MitoGraph/)** for mitochondria segmentation

- **[ChimeraX](https://www.cgl.ucsf.edu/chimerax/)** for tracking visualization

The following python packages are needed:

- **numpy**

- **scipy**

- **pandas**

- **igraph**

- **fastdist**

To install all packages, open command line and go to the root directory of MitoTNT:  

``
pip install -r python_dependencies.txt
``


In [1]:
import os
import numpy as np
import pandas as pd
import igraph as ig
from tqdm.notebook import trange
from mitotnt import generate_tracking_inputs, network_tracking, tracking_visualization, detect_fusion_fission, post_analysis

# update functions everytime after changes made
%load_ext autoreload
%autoreload 2

# Network Tracking
## 1. Generate inputs for tracking
**In this section we will process the raw data into a format that is used for the subsequent tracking.**

First specify the directories we will use:
- `work_dir`: the directory where data will be processed and stored. For test data, you can use the directory of `test_data` on your machine
- `data_dir`: the directory where MitoGraph segmented mitochondria is stored. For test data, this is `test_data/mitograph`.
- `input_dir`: the directory where the processed inputs used for tracking will be stored. For test data, you can use `test_data/tracking_inputs`. This is an empty folder that will be created.

After specifying the folders, we need to set a few parameters:
- `start_frame`, `end_frame`: the range of frames to process.
- `frame_interval`: frame interval for the movie in seconds
- `node_gap_size`: the number of nodes to skip when creating full-resolution graphs from mitograph `.gnet` files. Default to 0 (use all nodes).

All processed inputs will be saved as a single compressed `.npz` file in `input_dir`.

In [3]:
# specify your desired directories (backslash in file directory in Windows needs to be converted to forward slash)
work_dir = 'D:/Github/MitoTNT/test_data/'
data_dir = work_dir+'mitograph/'
input_dir = work_dir+'tracking_inputs/'
if not os.path.isdir(input_dir):
    os.mkdir(input_dir)

# params for the test data
start_frame = 0
end_frame = 20
frame_interval = 3.253

print('Tracking frame {} to {}'.format(start_frame, end_frame))

Tracking frame 0 to 20


In [ ]:
# run the function
generate_tracking_inputs.generate(data_dir, input_dir,
                                  start_frame, end_frame,
                                  node_gap_size=0)

## 2. Frame-to-frame tracking
**In this section we will perform node assignments for each consecutive frames.**

In addition to the directories declared above, we will create `output_dir` to store the tracking outputs. For test data, you can use `test_data/tracking_outputs`.

Additional parameters needed for frame-to-frame tracking:
- `tracking_interval`: the frame interval between the two frames to be tracked. Default to 1 (every consecutive frame).  

Distance threshold is computed as the minimum of two values: 1) the distance to the N-th closest neighbor where N is given by `cutoff_num_neighbor`, default to 10; 2) the frame interval times the maximum allowed speed given by `cutoff_speed`, default to `None` and estimated from the distance matrix.

- `graph_matching_depth`: the maximum level used for graph comparison. Default to 2 (usually sufficient).
- `dist_exponent`, `top_exponent`: the final cost term is given by D<sup>dist_exponent</sup> x T<sup>top_exponent</sup>, where D, T are the distance and topology costs respectively. Default both to 1 (equal weighting).


In [14]:
# specify additional directories
output_dir = work_dir+'tracking_outputs/'
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)
    
start_frame = 0
end_frame = 20
tracking_interval = 1

In [15]:
# run the function
network_tracking.frametoframe_tracking(input_dir, output_dir, start_frame, end_frame, frame_interval, tracking_interval,
                                       cutoff_num_neighbor=8, cutoff_speed=None,
                                       graph_matching_depth=2, dist_exponent=1, top_exponent=1)

Data loading ...



Tracking in progress:   0%|          | 0/20 [00:00<?, ?it/s]


Start tracking frame 0 and 1 ...
Distance cost matrix takes 0.88 s
Topology cost matrix takes 2.77 s
Number of nodes at frame 0, 1 are 1539, 1743
Number of nodes linked, terminated at frame 0: 1206, 333
Number of nodes initiated at frame 1: 537
Mean, std of distances for tracked nodes: 0.264843, 0.165531
Average cost: 6.4
Tracking is complete and takes 8.33 s


Start tracking frame 1 and 2 ...
Distance cost matrix takes 0.99 s
Topology cost matrix takes 3.22 s
Number of nodes at frame 1, 2 are 1743, 1714
Number of nodes linked, terminated at frame 1: 1388, 355
Number of nodes initiated at frame 2: 326
Mean, std of distances for tracked nodes: 0.233727, 0.172812
Average cost: 7.7
Tracking is complete and takes 6.10 s


Start tracking frame 2 and 3 ...
Distance cost matrix takes 1.01 s
Topology cost matrix takes 2.97 s
Number of nodes at frame 2, 3 are 1714, 1766
Number of nodes linked, terminated at frame 2: 1398, 316
Number of nodes initiated at frame 3: 368
Mean, std of distances for

## 3. Gap closing
**In this section we attempt to merge tracks that are mistakenly terminated during frame-to-frame tracking.**

Additional parameters need to be set:
- `min_track_size`: the minimum number of frames for the tracks to be kept. Default to 5.
- `max_gap_size`: the maximum number of frames for which gap closing is allowed. Default to 3. Value of 1 indicates no gap closing.
- `block_size_factor`: values less than 1 allows the sliding block implementation of gap closing to prevent memory overflow due to large cost matrix. The size of the block is given by `block_size_factor` * total number of tracks to be closed. Default to 1 (close all tracks at once).

The final node trajectories are saved in `final_node_tracks.csv` file.  
Each row is one node at one time point.  
Each column is an attribute of the given node, described below.

In [ ]:
network_tracking.gap_closing(input_dir, output_dir, start_frame, end_frame, tracking_interval,
                             min_track_size=4, max_gap_size=3, block_size_factor=1)

## 4. Evaluate output

The final node trajectories are saved in final_node_tracks.csv file.  
Each row is one node at one time point.  
Each column is an attribute of the given node, described below.  

### Columns

- `frame_id`: frame number of the node.
- `frame_node_id`: node id at the given frame. Each frame has its own indexing.
- `unique_node_id`: node id shared by all the nodes in the same track at different frames. Each track is uniquely indexed throughout the whole trajectory. This is essetially the tracking information.
- `frame_seg_id`: segment id for all the nodes in the same segment. Each frame has its own indexing. The branching points are not assigned.
- `frame_frag_id`: fragment id for all the nodes in the same connected component. Each frame has its own indexing.
- `connected_unique_node_id`: unique_node_id for neigboring nodes in the graph. This has all the topology information.
- `x`, `y`, `z`: coordinates for the node.
- `intensity`, `width`: pixel intensity and tubular width for the node given by MitoGraph.

# Visualization
**In this section we will visualize the tracked mitochondrial networks in ChimeraX**

**Please first download [ChimeraX](https://www.cgl.ucsf.edu/chimerax/)**

We need to specify the directory to save visualization files
- `vis_dir`: store `.cxc` commands to load in ChimeraX. You can use `work_dir/chimerax_visualization/` for example.
- `vis_data_dir`: store `.cmap ` and `.bild` files created for each frame and used for visualization. You can use `vis_dir/data/` for example.

In [5]:
# specify the directory for storing processed data
vis_dir = work_dir+'chimerax_visualization/'
vis_data_dir = vis_dir+'data/'
if not os.path.isdir(vis_dir):
    os.mkdir(vis_dir)
if not os.path.isdir(vis_data_dir):
    os.mkdir(vis_data_dir)

# visualizing tracks can take some time, recommend to start with a few frames
start_frame = 0
end_frame = 5

## 1. Transform .tif to match MitoGraph coordinates (optional)
Because MitoGraph does coordinate transformation, original `.tif` files need to be transformed.\
This is only needed if you want to show fluorescence cloud when visualizing tracking.
- `voxel_size`: provide the voxel_size same as inputs for MitoGraph segmentation, in the format of 'x_size,y_size,z_size'.  
For example, `voxel_size='0.2,0.2,0.4'` refers to lateral pixel size 0.2 μm and axial pixel size 0.4 μm.

**After the output generate_transformed_tif.cxc file is created, load it in ChimeraX to save the transformed images.**

In [6]:
tracking_visualization.generate_transformed_tif(data_dir, vis_dir, vis_data_dir,
                                                start_frame, end_frame, tracking_interval, 
                                                voxel_size='0.145,0.145,0.145')

Load file D:/Github/MitoTNT/test_data/chimerax_visualization/generate_transformed_tif.cxc 
in ChimeraX to generate transformed tif files for proper visualization


## 2. Create ChimeraX rendering of the skeleton (optional but recommended)
We can use MitoGraph-generated `*skeleton.vtk` files for visualizing skeleton, but this is not ideal because it has fixed width and color.\
Alternatively here, we can render the skeleton using BILD format in ChimeraX. This allows us to set the skeleton sizes, node sizes and color. However, it also takes much longer to load in ChimeraX.
- `skeleton_colors`: a list of two colors to render for current and next frames. We use blue for current frame and red for next frame. [See more colors.](https://www.cgl.ucsf.edu/chimerax/docs/user/commands/colornames.html)
- `skeleton_size`: diameter of the skeleton that connects nodes.
- `node_size`: diameter of the spheres that make up the nodes. If `node_size`= `skeleton_size`, the nodes are not visible (but still required to fill the gaps along the skeletons).

In [7]:
tracking_visualization.generate_chimerax_skeleton(input_dir, vis_dir, vis_data_dir,
                                                  start_frame, end_frame, tracking_interval,
                                                  skeleton_colors=['blue','red'],
                                                  skeleton_size=0.035, node_size=0.035)

Generate network skeleton in BILD format ...
Color blue


  0%|          | 0/6 [00:00<?, ?it/s]

Color red


  0%|          | 0/6 [00:00<?, ?it/s]

Done


## 3. Create ChimeraX rendering of tracking vectors
We will use the frame-to-frame node assignments to draw the tracking vectors for two frames.
- `arrow_color`: color of the tracking arrows. Default to black.
- `arrow_size`: diameter of the tracking arrow stem.

In [8]:
tracking_visualization.generate_tracking_arrows(input_dir, output_dir, vis_data_dir, 
                                                start_frame, end_frame, tracking_interval,
                                                arrow_color='black', arrow_size=0.035)

Generate tracking arrows in BILD format ...


  0%|          | 0/5 [00:00<?, ?it/s]

Done


## 4. Visualize network tracking in ChimeraX
Now we can combine the visualization files created above to visualize the tracking of timelapse data.
- `show_tif`: if true include fluorescence cloud in background. 
- `tif_colors`: color of fluorescence cloud. See [colors](https://www.cgl.ucsf.edu/chimerax/docs/user/commands/colornames.html).
- `threshold_level`: if you want to change the contrast/thickness of the fluorescence cloud, use the [level argument](https://www.cgl.ucsf.edu/chimerax/docs/user/commands/volume.html) in ChimeraX.
- `use_chimerax_skeleton`:\
if true use BILD format skeleton which is more flexible but slower to load;\
if false use mitograph-generated .vtk files of fixed color and size (not recommended but quicker).
- `skeleton_colors`: same as in step 2.

**Open chimerax_visualization/visualize_tracking.cxc in ChimeraX. This may take some time. Click Home -> Backgound -> White to see it better.**

In [9]:
tracking_visualization.visualize_tracking(data_dir, input_dir, vis_dir, vis_data_dir,
                                          start_frame, end_frame, tracking_interval,
                                          show_tif=True, tif_colors=['deep sky blue','tomato'], threshold_level='', 
                                          use_chimerax_skeleton=True, skeleton_colors=['blue','red'])

Load file D:/Github/MitoTNT/test_data/chimerax_visualization/visualize_tracking.cxc 
in ChimeraX to visualize tracking


# Detect Remodeling Events
**In this section we will detect nodes that undergo fusion or fission events based on the tracking results.**

This is done using a sliding-window approach to identify nodes that undergo persistent structural changes as opposed to transient segmentation differences.  
First, the fragment indices for each node are recorded for the `half_win_size` frames before and after the current frame, to form the fragment list.  
Second, for each network edge, the fragment lists for the connected nodes are compared.  
Finally, Fission will be declared if the fragment lists before the current frame are strictly identical, as well as the fragment lists after the current frame are strictly non-overlapping. 
Since fusion events can be considered as fission events reversed in time, the opposite criterion is used for fusion detection.

Note because of the sliding window approach:
`start_frame` must be >= `half_win_size` and `end_frame` must be <= total number of frames - `half_win_size`

Please specify:

- `stride_size`: step size for sliding the window in number of frames. Default to 1 (to detect events happening in every frame).

- `half_win_size`: size of the half sliding window in number of frames. The higher the value the stricter the requirement for calling fusion/fission. Default to 4.

- `min_tracked_frames`: minimum number of frames that are tracked in both half window in order to declare an event. Default to 2.

In [ ]:
detect_fusion_fission.detect(input_dir, output_dir, start_frame=4, end_frame=17,
                             stride_size=1,
                             half_win_size=4,
                             min_tracked_frames=2)

The remodeling events are saved under `tracking_output/remodeling_events.csv`  
Multiple remodeling nodes located in proximity (less than 5 edges away) are grouped into a single fission/fusion site.
Columns in the output:

- `type`: fusion or fission

- `frame_id`: a single frame number for describing when the event happens

- `frame_id_before`: the frame numbers before the event for each detected node (may be different due to gap closing)

- `frame_id_after`: the frame numbers after the event for each detected node (may be different due to gap closing)

- `node_id_before`: the `frame_node_id` at corresponding frame before the event for each detected node

- `node_id_after`: the `frame_node_id` at corresponding frame after the event for each detected node

- `frag_id_before`: the `frame_frag_id` at corresponding frame before the event for each detected node

- `frag_id_after`: the `frame_frag_id` at corresponding frame after the event for each detected node

- `unique_node_id`: the `unique_node_id` for each detected node

`frame_node_id`, `frame_frag_id`, `unique_node_id` are as defined in the node tracking outputs


# Post Analysis
## Motility Measurements
**In this section we will use the tracking results to compute diffusivity at three levels of description and visualize motility in space.**

The diffusision coeffients can be computed for 1) nodes, 2) segments, 3) fragments, in the order of higher level of coarse graining.  

We need to specify the directory for save post analysis results:

- `analysis_dir`: umbrella directory for post analysis. A good choice is `work_dir+'post_analysis/'`. 

- `analy_motility_dir`: directory for saving motility measurements. This is usually a subfolder in `analysis_dir`, for example `analysis_dir+'motility/'`.  

In [10]:
# specify directories
analysis_dir = work_dir+'post_analysis/'
analy_motility_dir = analysis_dir+'motility/'
if not os.path.isdir(analysis_dir):
    os.mkdir(analysis_dir)
if not os.path.isdir(analy_motility_dir):
    os.mkdir(analy_motility_dir)

Information needed for computing MSD vs. time delay curve:

- `max_tau`: maximum number of frames/datapoints used for linear regression. Default to 5.

Additional information for computing segment and fragment level diffusivity:

- `selected_frames`: because segment and fragments undergo constant remodeling, we need to select the frames at which the segment and fragments are evaluated. This is recommended to be separated by 2x half window size (see below). Each frame will be visualized separately.

- `half_win_size `: the time window size (frames) before and after the selected center frames for collecting track coordinates. Default to 10.

- `tracked_ratio`: the minimum ratio of tracked nodes in each segment/fragment to be qualified for calculating diffusivity. Default to 0.5. 

### node diffusivity

In [11]:
post_analysis.compute_node_diffusivity(input_dir, output_dir, analy_motility_dir, frame_interval, max_tau=5)

Computing MSD for tracks ...


  0%|          | 0/2531 [00:00<?, ?it/s]

Fitting linear curve and computing diffusion coefficient ...


  0%|          | 0/2531 [00:00<?, ?it/s]

Data saved at D:/Github/MitoTNT/test_data/post_analysis/motility/node_diffusivity.csv


### segment diffusivity

In [ ]:
post_analysis.compute_segment_diffusivity(input_dir, output_dir, analy_motility_dir,
                                          frame_interval, max_tau=5, tracked_ratio=0.3, half_win_size=10, selected_frames=[10])

### fragment diffusivity

In [ ]:
post_analysis.compute_fragment_diffusivity(input_dir, output_dir, analy_motility_dir,
                                           frame_interval, max_tau=5, tracked_ratio=0.3, half_win_size=10, selected_frames=[10])

The data is saved in `analy_motility_dir/*diffusivity.csv` files. Each row is a node/segment/fragment. The columns are explained below.
### Columns

 - `center_frame_id`: selected frame for determining the segment/fragment diffusivity. N/A for node diffusivity.
 
 - `unique_node_id` as in `final_node_tracks.csv`. `seg_id`, `frag_id` are specific to `center_frame_id`.

 - `diffusivity`: slope of MSD vs. time delay curve divided by 6 to account for 3D random walk.

 - `msd`: MSD per frame, euqal to 6 x diffusivity x frame interval

 - `r_squared`: coefficient of determination for the linear regression.

 - `num_points`: number of points in MSD vs. time delay curve used for linear regression.

## Visualize motility
**In this section we will color the mitochondrial network by the local motility, again at three levels.**

In [12]:
post_analysis.map_node_motility_onto_surface(input_dir, output_dir, analy_motility_dir,
                                             node_size=0.2, selected_frames=[10])

1074 nodes are mapped out of total 1719 nodes

Load file D:/Github/MitoTNT/test_data/post_analysis/motility/map_node_motility_frame_10.bild 
in ChimeraX to visualize motility map


In [ ]:
post_analysis.map_segment_motility_onto_surface(input_dir, output_dir, analy_motility_dir,
                                                node_size=0.2, selected_frames=[10])

In [ ]:
post_analysis.map_fragment_motility_onto_surface(input_dir, output_dir, analy_motility_dir,
                                                 node_size=0.2, selected_frames=[10])